# ECO475 Group 2 Notebook Code

### Author: Shih-Chieh Lee, Lingyun Ma, Yuwen Zhao

# 1. Basic Setting

## a. Package Install

In [ ]:
#!pip install stats-can
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install statsmodels
#!pip install linearmodels
#!pip install tabula-py #Note: Pls install tabula-py, not tabula——血的教训
#!pip install warnings

## b. Package Import 

In [2]:
# Data Collection Packages
from stats_can import StatsCan #read StatsCan data 


sc = StatsCan(data_folder="/Users/changanlee/Documents/GitHub/Housing_Price_Immigration/Input") 
#Create an instance of StatsCan class

In [3]:
# Import tabula and check java environment
from tabula.io import read_pdf  #Scrape table from pdf files
import requests 
from datetime import datetime
import calendar
import re

In [4]:
# Data Processing Packages
import pandas as pd #pandas
import numpy as np 
import matplotlib.pyplot as plt #data visualization
%matplotlib inline
# activate plot theme
import qeds

In [5]:
# Stats Model Packages
import statsmodels.api as sm # statistical model
from statsmodels.iolib.summary2 import summary_col # summary table for regression result
from linearmodels.iv import IV2SLS # IV 

In [6]:
# Silence all the warnings cuz they're absolutely annoying if you loop it multiple times
import warnings
#warnings.filterwarnings('ignore')

# 2. Data Collection

## A. StatsCan Data 

Let's start with datasets from Statistics Canada, as it's earier to collect directly using StatsCan library

In [ ]:
CMA_pop = sc.table_to_df("17-10-0135-01")
CMA_pop.head()

## B. CREA Monthly House Price Index Data

### 1) Define Functions

In [36]:
# Define func to generate URLs based on months and years
def generate_url(month, year, cma):
    if cma == "Toronto":
        if month < 10:
            return f"https://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_0{month}{year}.pdf"
        else:
            return f"https://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_{month}{year}.pdf"
            #return the corresponding GTA HPI monthly report pdf for scrapping
    elif cma == "Vancouver":
        url = generate_url_van(month, year)
        return url

### 1) GTA Data

#### a) Define Functions

In [8]:
# Define func to extract table from pdf monthly report & data cleaning
def extract_table(url, month, year):

    # Extract table from pdf
    tables = read_pdf(url, pages="2", lattice = "True", multiple_tables = "False", 
                      area = [10,0,97,100],relative_area = "True", silent = "True") 
        # Note: lattice should be set as True for our case 
            #to read everything on page 2 as one table
            
    table = tables[0]
    
    # Clean the data table
    new_header = table.iloc[0] # Set new header with the first row of the table
    table = table[1:]  # Take the data below the header row
    table.columns = new_header  # Set the new header
    
    # We only want data about composite / residential property, which is the first four columns
    table = table.iloc[:, :4]
    
    # Rename the first column to "Location" 
    table = table.rename(columns={table.columns[0]: "Location"})

    # New Column for Month-Year
    if month < 10:
        table["Month_Year"] = f'0{month}_20{year}'
    else:
        table["Month_Year"] = f'{month}_20{year}'
        
    table.reset_index(drop=True, inplace=True)
    
    return table

#### b) Loop to Scrape Data

In [ ]:
data_list = []
cma = "Toronto"

for year in range(16, 23):
    if year != 22:
        for month in range(1, 13):
            url = generate_url(month, year, cma)
            extracted_data = extract_table(url, month, year)
        
            if extracted_data is not None:
                extracted_data.reset_index(drop = True, inplace = True)
                data_list.append(extracted_data)

    else:
        for month in range(1, 6):
            url = generate_url(month, year)
            extracted_data = extract_table(url, month, year)
        
            if extracted_data is not None:
                extracted_data.reset_index(drop = True, inplace = True)
                data_list.append(extracted_data)
            
HPI_GTA = pd.concat(data_list, ignore_index = True)

In [ ]:
HPI_GTA.to_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GTA.csv",
               index = False)

For some pdf formatting reasons that I cannot solve right now, the last monthly report that can be extracted using read_pdf is May 2022. May need to mannually extract the rest of the data. See example below:

In [ ]:
#Take June 2022 for example

url = generate_url(6,22) 
tables = read_pdf(url, pages = "2", lattice = "True", multiple_tables = "False", area = [10,0,97,100],relative_area = "True")
tables[0].head()

The column heading format is correct, but the value it reads is completely nonsense.

Whatever... Let's move on to GTA data first

### 2) GVA Data

First of all, Jan. 2016 ~ July 2016 monthly report is missing...

Second, GVA report url has multiple formats over time:

https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-August-2016.pdf

https://www.gvrealtors.ca/content/dam/rebgv_org_content/monthly-market-reports/2018-Dec-stats-pkg.pdf

https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/marketwatch/monthly_reports/REBGV-Stats-Pkg-November-2019-F.pdf

OK, let's try accessing the report using all 3 formats for all month-year combination and see how it goes

In [39]:
def generate_url_van(month, year):

    # Define the base URLs for each format
    base_urls = [
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/monthly-market-reports/{Year}-{Month}-stats-pkg.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/marketwatch/monthly_reports/REBGV-Stats-Pkg-{Month}-{Year}-F.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Package-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV-Stats-Pkg-{Month}-{Year}-Updated%20HPI.pdf",
        "https://members.rebgv.org/news/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{YearMonth}-REBGV-Stats-Pkg-{Month}-{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/REBGV%20Stats%20Package%20{Month}%20{Year}.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{Year}-01-{Month}-Stats-Package.pdf",
        "https://www.gvrealtors.ca/content/dam/rebgv_org_content/pdfs/monthly-stats-packages/{Year}-01-{Month}-Stats--Package.pdf"
    ]


    # Get the month name and abbreviation
    year = 2000 + year
    month_name = calendar.month_name[month]
    month_abbr = calendar.month_abbr[month]
    year_month = f"{year}{month:02d}"

    # Iterate through each URL pattern
    for base_url in base_urls:
        # Replace placeholders in the URL pattern
        url_1 = base_url.format(Year=str(year), Month=month_name, YearMonth=year_month)
        url_2 = base_url.format(Year=str(year), Month=month_abbr, YearMonth=year_month)

        # Try fetching the first URL
        if requests.get(url_1).status_code == 200:
            return url_1

        # Try fetching the second URL if the first one fails
        elif requests.get(url_2).status_code == 200:
            return url_2

        # Special case for September URLs
        if month == 9:
            url_3 = base_url.format(Year=str(year), Month="Sept", YearMonth=year_month)
            if requests.get(url_3).status_code == 200:
                return url_3

In [28]:
def extract_table_van(url, month, year):    
    tables = read_pdf(url, pages = "3", multiple_tables = "False")
    
    if not tables or tables[0].empty:
        return None
    
    else:
        table = tables[0]
        new_header = table.iloc[1] # Set new header with the second row of the table
        new_header[0] = "Property Type"
        new_header[1] = "Location"
        table = table[2:]  # Take the data below the header row
        table.columns = new_header

        end_here_index = (table['Property Type'] == 'Single Family Detached').idxmax()
        table = table.iloc[:end_here_index, 1:]

        if month < 10:
            table["Month_Year"] = f'0{month}_{year}'
        else:
            table["Month_Year"] = f'{month}_{year}'

    return table

In [38]:
data_list = []
cma = "Vancouver"

for year in range(16, 24):
    for month in range (1,13):
        
        url = generate_url(month, year, "Vancouver")
        if url is not None:
            extracted_data = extract_table_van(url, month, year)

            if extracted_data is not None:
                extracted_data.reset_index(drop=True, inplace=True)
                data_list.append(extracted_data)
                print("Collected", month,"-",year, " data.")

HPI_GVA = pd.concat(data_list, ignore_index=True)

columns_name = ["Location", "Price", "Index", "%1M_Change", "%3M_Change", "%6M_Change", 
               "%1Y_Change", "%3Y_Change", "%5Y_Change", "%10Y_Change", "Month_Year"]

HPI_GVA.columns = columns_name

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 3 - 16  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 4 - 16  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


...

Collected 9 - 23  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 10 - 23  data.


/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[0] = "Property Type"
/var/folders/hn/gfxxh51d76nbszjdcrp5pzy80000gn/T/ipykernel_47560/3038214230.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_header[1] = "Location"


Collected 11 - 23  data.


In [33]:
data_list

[]

In [ ]:
HPI_GVA.to_csv("/Users/changanlee/Desktop/University/Undergrad/4th-Year/Winter Semester/ECO475/Term Paper/Raw Data/HPI_GVA.csv",
               index = False)